Data Extraction

In [1]:
#step1: Data Extraction
#Read data From the csv file into a DataFrame

import pandas as pd

In [2]:
# Define the CSV file path
csv_file = 'E:\\BIG DATA BY ineuron\\PuntPartenerTestAssignment\\ASSIGNMENT\\Updated_sales_data.csv'

In [3]:
## Extract: Read data from CSV into a DataFrame
df = pd.read_csv(csv_file)

In [4]:
#check data Extraction
df.head(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04-07-2019 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04-12-2019 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04-12-2019 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"


Data Transformation

In [5]:
# Step 2: Data Transformation
#Error : some value null and str.
#Error Solve: Convert to numeric Value

df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'],errors='coerce')

In [6]:
#Error : some value null and str.
#Error Solve: Convert to numeric Value

df['Price Each'] = pd.to_numeric(df['Price Each'],errors='coerce')

In [7]:
# Calculate total sales, extract year and month, and aggregate data
# Transform: Calculate total sales for each order
df['total_sales'] = df['Quantity Ordered'] * df['Price Each']

In [8]:
#Error Found:Unknown string format: Order Date present at position 518
#Error solve: format parameter to specify the expected date format

df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M', errors='coerce')


In [9]:
# Transform: Extract year and month from the order_date
df['Order Year'] = df['Order Date'].dt.year

In [10]:
df['Order Month'] = df['Order Date'].dt.month

In [11]:
# Aggregate: Calculate total sales for each product in each month
product_sales = df.groupby(['Product','Order Year','Order Month'])['total_sales'].sum().reset_index()

Data Loading

In [12]:
# Step 3: Data Loading
# Create a SQL database (SQLite) and define tables to store the transformed data
# Load the data into these tables


import sqlite3

In [13]:
# Load: Create a SQLite database and a connection
conn = sqlite3.connect('sales_data.db')

In [14]:
# Load: Define tables and load data into them
df.to_sql('orders', conn, index=False, if_exists='replace')
product_sales.to_sql('product_sales', conn, index=False, if_exists='replace')


38

In [15]:
# Commit and close the connection
conn.commit()
conn.close()

SQL Queries

In [17]:

# 1. What was the total sales amount for each product in the last quarter (last 3 months)?query1 = """
query1 = """
SELECT product, SUM(total_sales) as total_sales
FROM product_sales
WHERE order_year = (SELECT MAX(Order) FROM product_sales)
AND order_month >= (SELECT MAX(order_month) FROM product_sales) - 2
GROUP BY product;
"""

In [19]:
# 2. List the top 5 products by total sales amount for the entire dataset.
query2 = """
SELECT product, SUM(total_sales) as total_sales
FROM product_sales
GROUP BY product
ORDER BY total_sales DESC
LIMIT 5;
"""

In [20]:
# 3. Calculate the monthly average sales for each product over the entire dataset.
query3 = """
SELECT product, AVG(total_sales) as avg_sales
FROM product_sales
GROUP BY product;
"""

Data Validation

In [21]:
# Step 5: Data Validation and Error Handling (Bonus)

# Data Validation: Check for missing values in essential columns
missing_values = df.isnull().sum()
if any(missing_values):
    pass
    # Handle missing values appropriately (e.g., log and/or skip rows)

    # Data Validation: Check for anomalies in data (negative quantities or prices)
anomalies = df[(df['Quantity Ordered'] < 0) | (df['Price Each'] < 0)]
if not anomalies.empty:
    pass
    # Handle anomalies appropriately (e.g., correct values, log issues)

# Error Handling: Handle exceptions during the ETL process
try:
    # Code for ETL
    pass 
except FileNotFoundError as e:
    # Log the error and exit gracefully
    print(f"Error: {e}")
except Exception as e:
    # Handle other exceptions and log appropriately
    print(f"Error: {e}")